In [ ]:
# Import des bibliothèques nécessaires
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# 1. Chargement du dataset Iris
# Le dataset Iris est un jeu de données classique contenant trois classes de fleurs
# Chaque fleur est décrite par 4 caractéristiques (longueur/surface des pétales et sépales)
iris = load_iris()
X = iris.data  # Les caractéristiques d'entrée
y = iris.target  # La variable cible (les classes de fleurs)

# 2. Division des données en ensemble d'entraînement et de test
# On divise les données en 70% pour l'entraînement et 30% pour le test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. Création du modèle XGBoost
# Le modèle XGBoost est initialisé pour une tâche de classification multiclasse
# 'use_label_encoder' est mis à False pour désactiver l'encodage d'étiquettes déprécié
# 'eval_metric' est défini sur 'mlogloss' (logarithme de la perte pour classification multiclasse)
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# 4. Entraînement du modèle sur les données d'entraînement
model.fit(X_train, y_train)

# 5. Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# 6. Calcul de la précision du modèle
# L'accuracy mesure la proportion de prédictions correctes sur les données de test
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle : {accuracy * 100:.2f}%")

# 7. Visualisation de l'importance des caractéristiques
# XGBoost permet de visualiser l'importance relative des caractéristiques
xgb.plot_importance(model)
plt.show()

# 8. Réglage des hyperparamètres avec GridSearchCV pour améliorer le modèle
# On utilise GridSearchCV pour trouver les meilleurs hyperparamètres pour notre modèle
from sklearn.model_selection import GridSearchCV

# Définir une grille de paramètres à tester
param_grid = {
    'max_depth': [3, 5, 7],  # Profondeur maximale des arbres
    'n_estimators': [50, 100, 200],  # Nombre d'arbres (estimators)
    'learning_rate': [0.01, 0.1, 0.2]  # Taux d'apprentissage (learning rate)
}

# Création de GridSearchCV pour trouver les meilleurs paramètres
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')

# Entraîner GridSearchCV sur les données d'entraînement
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres trouvés
print(f"Meilleurs paramètres : {grid_search.best_params_}")

# Utiliser le meilleur modèle trouvé pour faire des prédictions
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

# Calculer la précision du meilleur modèle
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Précision du meilleur modèle : {accuracy_best * 100:.2f}%")

# 9. Sauvegarde du modèle XGBoost
# Une fois que nous avons un modèle performant, nous pouvons le sauvegarder pour l'utiliser plus tard
model.save_model("xgboost_model.json")

# 10. Chargement du modèle sauvegardé pour l'utiliser sans avoir besoin de réentraîner
# On peut recharger un modèle précédemment sauvegardé
loaded_model = xgb.XGBClassifier()
loaded_model.load_model("xgboost_model.json")

# On peut maintenant utiliser 'loaded_model' pour faire des prédictions directement
y_loaded_pred = loaded_model.predict(X_test)
accuracy_loaded = accuracy_score(y_test, y_loaded_pred)
print(f"Précision du modèle chargé : {accuracy_loaded * 100:.2f}%")
